Importing libraries

In [1]:
import cv2
import random
import mediapipe as mp

Declaring Global Variables

In [15]:
rad = 5
radiusImcrement = 3 
pos = [100,100]
OnOff = 1 # -1 means ball is not drawing and 1 means ball is drawing
timer = 0
timer1 = 0
stumpsTopLeft = (300,300);
stumpsBottomRight = (380,480);
color = (0,0,255)
Xdirection = 640
Ydirection = 480

Functions

In [3]:
def drawBall(img,centerTuple,radi, color)->None:
    cv2.circle(img,centerTuple,radi,color,cv2.FILLED)

def generateRandomCordinates():
    return (random.randint(0,500),random.randint(0,450))

def reset(onOff,rad):
    pos[0],pos[1] = generateRandomCordinates()
    onOff = (-1)*onOff
    pass

def drawStumps(img, topLeft, bottomRight):
    cv2.rectangle(img,topLeft,bottomRight, (0,255,0), 5)

def checkInStumps(centreTuple, onOff, topLeft, bottomRight):
    if onOff == -1:
        if centreTuple[0] > topLeft[0] and centreTuple[0] < bottomRight[0]:
            if centreTuple[1] > topLeft[1] and centreTuple[1] < bottomRight[1]:
                return True
    return False

def averageOfHand(handLms, img):
    sumX = 0
    sumY = 0
    for i in [5,9,13,17,0]:
        sumX = sumX + handLms.landmark[i].x
        sumY = sumY + handLms.landmark[i].y
    sumX = sumX/5
    sumY = sumY/5
    cv2.circle(img,(int(sumX*640),int(sumY*480)),5,(0,0,255),cv2.FILLED)
    return (int(sumX*640),int(sumY*480))

def checkForBall(handCenterTuple, onOff, ballCenter, ballRadius):
    if onOff == 1:
        if handCenterTuple[0] > ballCenter[0] - ballRadius and handCenterTuple[0] < ballCenter[0] + ballRadius:
            if handCenterTuple[1] > ballCenter[1] - ballRadius and handCenterTuple[1] < ballCenter[1] + ballRadius:
                return True
    return False
    

In [4]:
hands = mp.solutions.hands
hand = hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)
drawFunc = mp.solutions.drawing_utils


In [16]:
cap = cv2.VideoCapture(0)
cap.set(3, Xdirection)
cap.set(4, Ydirection)
score = 0
point = 0
while True:
    success, img = cap.read()
    img = cv2.flip(img, 1)
    
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
    resultsHand = hand.process(imgRGB) 
    if resultsHand.multi_hand_landmarks:
        for handLms in resultsHand.multi_hand_landmarks:
            try:
                xVal , yVal = averageOfHand(handLms,img)
                catch = True 
            except:
                catch = False
            
            if catch==False:
                xVal, yVal = 0,0
            if(checkForBall((xVal,yVal),OnOff,pos,rad)): 
                print(score)
                color = (0,255,0)
                rad = 8
                OnOff = -1
                point = 48
            else:
                color = (0,255,255)
                
            if(point>3):
                point = point -4
            
            cv2.putText(img,'Score '+ str(score), (10, 70), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 0), 3)
            cv2.putText(img,'Time '+ str(point), (10, 100), cv2.FONT_HERSHEY_PLAIN, 3, (0, 255, 0), 3)
            if(point>0):
                if(checkInStumps((xVal,yVal),OnOff,stumpsTopLeft,stumpsBottomRight)):
                        score = score + 1
                        print("HIT")
                        point = 0

            drawFunc.draw_landmarks(img, handLms, mp.solutions.hands.HAND_CONNECTIONS)

    rad = rad + radiusImcrement
    if OnOff==1:
        drawBall(img,(pos[0],pos[1]),int(rad),color)
    if rad>35:
        radiusImcrement = -1*(radiusImcrement)
    if rad<5:
        radiusImcrement = -1*(radiusImcrement)
        OnOff = (-1)*(OnOff)
        reset(onOff=OnOff,rad=rad)

    drawStumps(img,stumpsTopLeft,stumpsBottomRight)
    cv2.imshow("Video", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

0
HIT
1
1
1
1
1
1
HIT
